In [2]:
sys.path.append("..")
from kedro.config import ConfigLoader

In [4]:
from src.service.twitter.user_service import UserService
from src.service.twitter.tweet_service import TweetService
from src.util.twitter.thread_util import ThreadUtil

In [5]:
conf_loader = ConfigLoader("../conf", env="local")
bearer_token = conf_loader["credentials"]["twitter_api"]["authentication_tokens"]["bearer_token"]
username = "IAmPascio"


In [6]:
twitter_user = UserService(bearer_token).get_user(username)

In [7]:
tweets = TweetService(bearer_token).get_users_available_tweets(twitter_user)

In [8]:
thread_util = ThreadUtil(twitter_user, tweets)
thread_candidates = thread_util.get_users_threads_from_api()

In [29]:
df = thread_util.convert_tweets_to_df(thread_candidates)

In [30]:
user_id = str(twitter_user.user_id)

In [31]:
df = df[(df["in_reply_to_user_id"] == "None") | (df["in_reply_to_user_id"] == user_id)]

In [32]:
df = df.set_index("id")
df['conversation_id'] = df['conversation_id'].astype(str)
val_counts = df['conversation_id'].value_counts() > 2
df_thread = df.iloc[df.index.isin(val_counts[val_counts==True].index)]

In [33]:
df_thread.loc[:, ["created_at", "text", "retweet_count", "reply_count", "like_count", "quote_count", "impression_count"]]

,created_at,text,retweet_count,reply_count,like_count,quote_count,impression_count
id,,,,,,,
1618881501025943553,2023-01-27 08:00:01+00:00,ChatGPT is the future.\n\nBut most people are ...,2220,199,8139,36,659137
1618599199549034496,2023-01-26 13:18:15+00:00,"The past month, I've made over $13K online sel...",5,6,55,1,11279
1618587061602381824,2023-01-26 12:30:02+00:00,In the last 400 days:\n\n→Launched +50 product...,34,33,177,3,43594
1618519126598897667,2023-01-26 08:00:05+00:00,Have you heard of Notion: The Game?\n\nWorlds ...,81,922,484,6,111195
1618247324353925121,2023-01-25 14:00:02+00:00,"If every book caught on fire tomorrow, these a...",50,12,175,1,56830
...,...,...,...,...,...,...,...
1556996251106910208,2022-08-09 13:30:08+00:00,"I'm just 24 years old and I made over $7,000 o...",3,11,52,1,0
1556588576829329411,2022-08-08 10:30:11+00:00,The one app that you can't live without is ___...,1,23,12,0,0
1556206289470033921,2022-08-07 09:11:06+00:00,I made $130 by sending a 6 day email sequence....,87,125,107,2,0


In [28]:
df_thread.to_csv("../data/01_raw/IAmPascio_api_v2_version2.csv", index=True)

## Premium search

In [35]:
import tweepy
import pandas as pd
conf_loader = ConfigLoader("../conf", env="local")
consumer_key = conf_loader['credentials']['twitter_api']['consumer_keys']['api_key']
consumer_secret = conf_loader['credentials']['twitter_api']['consumer_keys']['api_key_secret']
access_token = conf_loader['credentials']['twitter_api']['authentication_tokens']['access_token']
access_token_secret = conf_loader['credentials']['twitter_api']['authentication_tokens']['secret_token']

In [36]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [132]:
results = api.search_full_archive(label="Development", query="from:IAmPascio", fromDate="202207271340", toDate="202208040000", maxResults=100)

In [133]:
list_of_tweets = []
for result in results:
    list_of_tweets.append([str(result.id),
                            result.created_at,
                            result.text,
                            str(result.in_reply_to_status_id),
                           result.retweet_count,
                           result.reply_count,
                           result.favorite_count,
                           result.quote_count, 0])

In [134]:
list_of_tweets[-1]

['1552466447347105792',
 datetime.datetime(2022, 7, 28, 1, 30, 19, tzinfo=datetime.timezone.utc),
 "7 months ago I didn't know:\n\n- Twitter\n- Creator economy\n- Creator economy on Twitter\n\n7 months later:\n\n- Have over… https://t.co/xHMswlMrcf",
 'None',
 0,
 8,
 27,
 0,
 0]

In [135]:
df = pd.DataFrame(list_of_tweets,
                  columns=['id', 'created_at', 'text', 'in_reply_to_status_id', "retweet_count", "reply_count", "like_count", "quote_count", "impression_count"])

In [96]:
df.to_csv("../data/01_raw/tweets_api_v1_1_2022_05_14.csv", index=False)

In [136]:
unique_ids = df['in_reply_to_status_id'].unique()

In [137]:
df = df.set_index("id")

In [138]:
df

,created_at,text,in_reply_to_status_id,retweet_count,reply_count,like_count,quote_count,impression_count
id,,,,,,,,
1554847764542095362,2022-08-03 15:12:49+00:00,@Jarodxu7 @ProductHunt Inspiring! Thanks for s...,1554843763092365313,0,0,0,0,0
1554833882746236930,2022-08-03 14:17:39+00:00,@AtulACleaver People don't realize how long 15...,1554830529429905408,0,0,0,0,0
1554821929009352704,2022-08-03 13:30:09+00:00,I've been on Twitter for over half a year now....,None,0,3,21,0,0
1554816407594057728,2022-08-03 13:08:13+00:00,@productive_kid Im rooting for you!,1554750480756219904,0,0,0,0,0
1554816358084485122,2022-08-03 13:08:01+00:00,@craigcherlet The first place is currently sha...,1554642249203519489,0,1,2,0,0
...,...,...,...,...,...,...,...,...
1552601639210598402,2022-07-28 10:27:31+00:00,@thenotionbar @capt_nandu @NotionHQ I’m workin...,1552005900008923136,1,0,1,0,0
1552539688476590080,2022-07-28 06:21:21+00:00,"@RyanHeng360 Thank you Ryan! A bit cheesy, but...",1552469740966264834,0,1,1,0,0
1552539571753308161,2022-07-28 06:20:53+00:00,@AtulACleaver Appreciate it Atul🚀,1552481855907442695,0,0,2,0,0


In [139]:
df_thread = df.iloc[df.index.isin(unique_ids)]

In [140]:
df_thread = df_thread.iloc[(df_thread['in_reply_to_status_id'] == "None").values, :]

In [141]:
df_thread

,created_at,text,in_reply_to_status_id,retweet_count,reply_count,like_count,quote_count,impression_count
id,,,,,,,,
1554459535447826435,2022-08-02 13:30:08+00:00,My first money made online came from a $0 prod...,None,4,9,44,3,0
1553372406303055878,2022-07-30 13:30:16+00:00,What's the best tweet you wrote this week? 👀\n...,None,0,6,6,0,0
1553010026033430532,2022-07-29 13:30:18+00:00,Growing on Twitter becomes easier after your f...,None,1,8,27,4,0


Add df_thread to df_all

In [477]:
df_all = df_all.append(df_thread)

/tmp/ipykernel_25213/1404768436.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df_thread)


In [460]:
# df_all = df_thread.copy()

In [478]:
df_all

,created_at,text,in_reply_to_status_id,retweet_count,reply_count,like_count,quote_count,impression_count
id,,,,,,,,
1552285211458736128,2022-07-27 13:30:09+00:00,The greatest productivity app of ALL TIME:\n\n...,None,19,14,214,11,0
1551205611911651329,2022-07-24 14:00:12+00:00,"I saw a guy ask his audience, ""What’s the best...",None,7,5,21,1,0
1550480849497477120,2022-07-22 14:00:15+00:00,What's my goal?\n\n1. Building an online busin...,None,0,8,28,0,0
1550400324812292096,2022-07-22 08:40:17+00:00,@NotionHQ super users:\n\nIs there are way to ...,None,0,2,3,0,0
1550118442195230722,2022-07-21 14:00:11+00:00,Twitter is a goldmine of knowledge.\n\nBut 97....,None,1,3,13,0,0
...,...,...,...,...,...,...,...,...
1468686902840774671,2021-12-08 21:00:19+00:00,Are you ready for a new @NotionHQ Template?\n\...,None,2,4,7,0,0
1468217057049202688,2021-12-07 13:53:19+00:00,🚀 I'm 23 years old and I just launched my firs...,None,0,2,8,0,0
1467107750773669889,2021-12-04 12:25:19+00:00,⚠️ LAUNCH ALERT ⚠️\n\nMy first-ever template:\...,None,13,3,25,4,0


In [479]:
df_all.to_csv("../data/01_raw/all_threads_v1_1.csv", index=True)

Transform Links into hooks:

In [109]:
df = pd.read_csv("../data/01_raw/IAmPascio_all_threads_v1_1.csv")
client = tweepy.Client(bearer_token)

In [116]:
tweets = client.get_tweets(list(df['id'].values), tweet_fields="id,created_at,text,conversation_id,in_reply_to_user_id,public_metrics")

In [126]:
full_texts = [el.text for el in tweets[0]]

In [129]:
df["text"] =full_texts

In [131]:
df.to_csv("../data/01_raw/IAmPascio_all_threads_v1_1.csv", index=False)